## 基于模式匹配的对话机器人

效果
```
Pattern: (我想要A)
Response: (如果你有 A，对你意味着什么呢？)

Input: (我想要度假)
Response: (如果你有度假，对你意味着什么呢？)
```

In [62]:
# 定义占位符格式 ?+字母
def is_variable(pat):
    return pat.startswith('?') and all(s.isalpha() for s in pat[1:])

In [63]:
# 语句列表和模式列表是否匹配
def pat_match(pattern, saying):
    if is_variable(pattern[0]): return True
    else:
        if pattern[0] != saying[0]: return False
        else:
            return pat_match(pattern[1:], saying[1:])

### 获得匹配的变量

以上的函数能够判断两个 pattern 是不是相符，但是我们更加希望的是获得每个variable对应的是什么值。

我们对程序做如下修改:

In [64]:
# 语句列表和模式列表是否匹配，若是则返回匹配内容。支持识别一个占位符的对应关系
def pat_match(pattern, saying):
    if is_variable(pattern[0]):
        return pattern[0], saying[0]
    else:
        if pattern[0] != saying[0]: return False
        else:
            return pat_match(pattern[1:], saying[1:])

In [65]:
pattern = 'I want ?X'.split()
saying = "I want holiday".split()

In [66]:
pat_match(pattern, saying)

('?X', 'holiday')

In [67]:
pat_match("?X equals ?X".split(), "2+2 equals 2+2".split())# 不支持多个变量

('?X', '2+2')

In [68]:
# 语句列表和模式列表是否匹配，若是则返回匹配内容。支持识别多个占位符的对应关系
def pat_match(pattern, saying):
    if not pattern or not saying: return []
    
    if is_variable(pattern[0]):
        return [(pattern[0], saying[0])] + pat_match(pattern[1:], saying[1:])
    else:
        if pattern[0] != saying[0]: return []
        else:
            return pat_match(pattern[1:], saying[1:])

In [69]:
pat_match("?X greater than ?Y".split(), "3 greater than 2".split())# 支持多个变量

[('?X', '3'), ('?Y', '2')]

#### 支持多语句的同名变量关联
如果我们知道了每个变量对应的是什么，那么我们就可以很方便的使用我们定义好的模板进行替换：

为了方便接下来的替换工作，我们新建立两个函数，一个是把我们解析出来的结果变成一个 dictionary，一个是依据这个 dictionary 依照我们的定义的方式进行替换。

In [70]:
# 占位符对应关系以字典存储
def pat_to_dict(patterns):
    return {k: v for k, v in patterns}

In [71]:
# 模板占位符根据字典取值
def subsitite(rule, parsed_rules):
    if not rule: return []
    
    return [parsed_rules.get(rule[0], rule[0])] + subsitite(rule[1:], parsed_rules)

In [72]:
got_patterns = pat_match("I want ?X".split(), "I want iPhone".split())

In [73]:
got_patterns

[('?X', 'iPhone')]

In [74]:
subsitite("What if you mean if you got a ?X".split(), pat_to_dict(got_patterns))

['What', 'if', 'you', 'mean', 'if', 'you', 'got', 'a', 'iPhone']

为了将以上输出变成一句话，也很简单，我们使用 Python 的 join 方法即可： 

In [75]:
john_pat = pat_match('?P needs ?X'.split(), "John needs resting".split())

In [76]:
' '.join(subsitite("What if you mean if you got a ?X".split(), pat_to_dict(got_patterns)))

'What if you mean if you got a iPhone'

In [77]:
john_pat = pat_match('?P needs ?X'.split(), "John needs vacation".split())

In [78]:
subsitite("Why does ?P need ?X ?".split(), pat_to_dict(john_pat))

['Why', 'does', 'John', 'need', 'vacation', '?']

In [79]:
' '.join(subsitite("Why does ?P need ?X ?".split(), pat_to_dict(john_pat)))

'Why does John need vacation ?'

那么如果我们现在定义一些patterns，就可以实现基于模板的对话生成了:

In [80]:
# 定义问答模板
defined_patterns = {
    "I need ?X": ["Image you will get ?X soon", "Why do you need ?X ?"], 
    "My ?X told me something": ["Talk about more about your ?X", "How do you think about your ?X ?"]
}

In [81]:
import random
def get_response(saying, rules):
    """" please implement the code, to get the response as followings:
    
    >>> get_response('I need iPhone') 
    >>> Image you will get iPhone soon
    >>> get_response("My mother told me something")
    >>> Talk about more about your monther.
    """
    # 找到提问对应问题模板
    # 提取问题的变量及值
    found_template = ''
    found_placeholders = []
    found_answers = []
    for question_template in rules.keys():
        placeholders = pat_match(question_template.split(), saying.split())
        if len(placeholders) != 0:
            found_template = question_template
            found_placeholders = placeholders
            found_answers = rules[question_template]
#     print(found_template)
#     print(found_placeholders)
#     print(found_answers)
    # 随机选择回答
    choice_answer = found_answers[random.randint(0,len(found_answers)-1)]
#     print(choice_answer)
    # 替换回答的变量值为问题中提取的变量值
    final_answer = subsitite(choice_answer.split(), pat_to_dict(found_placeholders))
#     print(final_answer)
    final_answer = ' '.join(final_answer)
    return final_answer

    
get_response('I need iPhone', defined_patterns)
get_response("My mother told me something", defined_patterns)

'How do you think about your mother ?'

### Segment Match

我们上边的这种形式，能够进行一些初级的对话了，但是我们的模式逐字逐句匹配的， "I need iPhone" 和 "I need ?X" 可以匹配，但是"I need an iPhone" 和 "I need ?X" 就不匹配了，那怎么办？ 

为了解决这个问题，我们可以新建一个变量类型 "?\*X", 这种类型多了一个星号(\*),表示匹配多个

首先，和前文类似，我们需要定义一个判断是不是匹配多个的variable

In [82]:
# 定义多单词匹配占位符格式 ?*+字母
def is_pattern_segment(pattern):
    return pattern.startswith('?*') and all(a.isalpha() for a in pattern[2:])

In [83]:
is_pattern_segment('?*P')

True

In [84]:
from collections import defaultdict

然后我们把之前的 ```pat_match```程序改写成如下， 主要是增加了 ``` is_pattern_segment ```的部分. 

In [85]:
fail = [True, None]

# 语句列表和模式列表是否匹配，若是则返回匹配内容。
# 支持识别多个占位符的对应关系
# 支持占位符匹配多个单词
def pat_match_with_seg(pattern, saying):
    if not pattern or not saying: return []
    
    pat = pattern[0]
    
    if is_variable(pat):
        return [(pat, saying[0])] + pat_match_with_seg(pattern[1:], saying[1:])
    elif is_pattern_segment(pat):
        match, index = segment_match(pattern, saying)
        return [match] + pat_match_with_seg(pattern[1:], saying[index:])
    elif pat == saying[0]:
        return pat_match_with_seg(pattern[1:], saying[1:])
    else:
        return fail

In [86]:
pat_match_with_seg('?*P is very good'.split(), "My dog and my cat is very good".split())

[('?P', ['My', 'dog', 'and', 'my', 'cat'])]

这段程序里比较重要的一个新函数是 ```segment_match```，这个函数输入是一个以 ```segment_pattern```开头的模式，尽最大可能进行，匹配到这个*边长*的变量对于的部分。

In [87]:
# 语句列表和模式列表是否匹配，若是则返回匹配内容。
# 支持识别一个占位符的对应关系
# 支持占位符匹配多个单词
# 支持返回匹配的多个单词数目
def segment_match(pattern, saying):
    seg_pat, rest = pattern[0], pattern[1:]
    seg_pat = seg_pat.replace('?*', '?')

    if not rest: return (seg_pat, saying), len(saying)    
    
    for i, token in enumerate(saying):
        if rest[0] == token and is_match(rest[1:], saying[(i + 1):]):
            return (seg_pat, saying[:i]), i
    
    return (seg_pat, saying), len(saying)

def is_match(rest, saying):
    if not rest and not saying:
        return True
    if not all(a.isalpha() for a in rest[0]):
        return True
    if rest[0] != saying[0]:
        return False
    return is_match(rest[1:], saying[1:])

In [88]:
segment_match('?*P is very good'.split(), "My dog and my cat is very good".split())

(('?P', ['My', 'dog', 'and', 'my', 'cat']), 5)

现在，我们就可以做到以下的匹配模式了: 

In [89]:
pat_match_with_seg('?*P is very good and ?*X'.split(), "My dog is very good and my cat is very cute".split())

[('?P', ['My', 'dog']), ('?X', ['my', 'cat', 'is', 'very', 'cute'])]

In [90]:
segment_match('?*P is very good and ?*X'.split(), "My dog is very good and my cat is very cute".split())

(('?P', ['My', 'dog']), 2)

如果我们继续定义一些模板，我们进行匹配，就能够进行更加复杂的问题了: 

In [91]:
# 定义问答模板
response_pair = {
    'I need ?X': [
        "Why do you neeed ?X"
    ],
    "I dont like my ?X": ["What bad things did ?X do for you?"]
}

In [92]:
pat_match_with_seg('I need ?*X'.split(), 
                  "I need an iPhone".split())

[('?X', ['an', 'iPhone'])]

In [93]:
subsitite("Why do you neeed ?X".split(), pat_to_dict(pat_match_with_seg('I need ?*X'.split(), 
                  "I need an iPhone".split())))

['Why', 'do', 'you', 'neeed', ['an', 'iPhone']]

 我们会发现，pat_to_dict在这个场景下会有有一点小问题，没关系，修正一些: 

In [94]:
def pat_to_dict(patterns):
    return {k: ' '.join(v) if isinstance(v, list) else v for k, v in patterns}

In [95]:
subsitite("Why do you neeed ?X".split(), pat_to_dict(pat_match_with_seg('I need ?*X'.split(), 
                  "I need an iPhone".split())))

['Why', 'do', 'you', 'neeed', 'an iPhone']

如果我们定义这样的一个模板:

In [96]:
("?*X hello ?*Y", "Hi, how do you do")

('?*X hello ?*Y', 'Hi, how do you do')

In [97]:
subsitite("Hi, how do you do?".split(), pat_to_dict(pat_match_with_seg('?*X hello ?*Y'.split(), 
                  "I am mike, hello ".split())))

['Hi,', 'how', 'do', 'you', 'do?']

### 现在是你的时间了

In [98]:
#我们给大家一些例子: 
    
rules = {
    "?*X hello ?*Y": ["Hi, how do you do?"],
    "I was ?*X": ["Were you really ?X ?", "I already knew you were ?X ."]
}

### 问题1

编写一个程序, ```get_response(saying, response_rules)```输入是一个字符串 + 我们定义的 rules，例如上边我们所写的 pattern， 输出是一个回答。

In [99]:
def get_response(saying, rules):
    # 找到提问对应问题模板
    # 提取问题的变量及值
    found_template = ''
    found_placeholders = []
    found_answers = []
    for question_template in rules.keys():
        placeholders = pat_match_with_seg(question_template.split(), saying.split())
#         print('placeholders=', placeholders)
#         if len(placeholders) != 0:
        if placeholders[0] != True: #!= fail
            found_template = question_template
            found_placeholders = placeholders
            found_answers = rules[question_template]
#     print(found_template)
#     print(found_placeholders)
#     print(found_answers)
    # 随机选择回答
    choice_answer = found_answers[random.randint(0,len(found_answers)-1)]
#     print(choice_answer)
    # 替换回答的变量值为问题中提取的变量值
    final_answer = subsitite(choice_answer.split(), pat_to_dict(found_placeholders))
#     print(final_answer)
    final_answer = ' '.join(final_answer)
    return final_answer


get_response('I was saying hello', rules)

'Were you really saying hello ?'

### 问题2

改写以上程序，将程序变成能够支持中文输入的模式。
*提示*: 你可以需用用到 jieba 分词

In [522]:
rule_responses = {
    '亿甲 hello 亿乙': ['How do you do', 'Please state your problem'],
    '亿甲 I want 亿乙': ['what would it mean if you got 亿乙', 'Why do you want 亿乙', 'Suppose you got 亿乙 soon'],
    '亿甲 if 亿乙': ['Do you really think its likely that 亿乙', 'Do you wish that 亿乙', 'What do you think about 亿乙', 'Really-- if 亿乙'],
    '亿甲 no 亿乙': ['why not?', 'You are being a negative', 'Are you saying \'No\' just to be negative?'],
    '亿甲 I was 亿乙': ['Were you really', 'Perhaps I already knew you were 亿乙', 'Why do you tell me you were 亿乙 now?'],
    '亿甲 I feel 亿乙': ['Do you often feel 亿乙 ?', 'What other feelings do you have?'],
    '亿甲你好亿乙': ['你好呀', '请告诉我你的问题'],
    '亿甲我想亿乙': ['你觉得亿乙有什么意义呢？', '为什么你想亿乙', '你可以想想你很快就可以亿乙了'],
    '亿甲我想要亿乙': ['亿甲想问你，你觉得亿乙有什么意义呢?', '为什么你想亿乙', '亿甲觉得... 你可以想想你很快就可以有亿乙了', '你看亿甲像亿乙不', '我看你就像亿乙'],
    '亿甲喜欢亿乙': ['喜欢亿乙的哪里？', '亿乙有什么好的呢？', '你想要亿乙吗？'],
    '亿甲讨厌亿乙': ['亿乙怎么会那么讨厌呢?', '讨厌亿乙的哪里？', '亿乙有什么不好呢？', '你不想要亿乙吗？'],
    '亿甲AI亿乙': ['你为什么要提AI的事情？', '你为什么觉得AI要解决你的问题？'],
    '亿甲机器人亿乙': ['你为什么要提机器人的事情？', '你为什么觉得机器人要解决你的问题？'],
    '亿甲对不起亿乙': ['不用道歉', '你为什么觉得你需要道歉呢?'],
    '亿甲我记得亿乙': ['你经常会想起这个吗？', '除了亿乙你还会想起什么吗？', '你为什么和我提起亿乙'],
    '亿甲如果亿乙': ['你真的觉得亿乙会发生吗？', '你希望亿乙吗?', '真的吗？如果亿乙的话', '关于亿乙你怎么想？'],
    '亿甲我?*z梦见亿乙':['真的吗? --- 亿乙', '你在醒着的时候，以前想象过亿乙吗？', '你以前梦见过亿乙吗'],
    '亿甲妈妈亿乙': ['你家里除了亿乙还有谁?', '嗯嗯，多说一点和你家里有关系的', '她对你影响很大吗？'],
    '亿甲爸爸亿乙': ['你家里除了亿乙还有谁?', '嗯嗯，多说一点和你家里有关系的', '他对你影响很大吗？', '每当你想起你爸爸的时候， 你还会想起其他的吗?'],
    '亿甲我愿意亿乙': ['我可以帮你亿乙吗？', '你可以解释一下，为什么想亿乙'],
    '亿甲我很难过，因为亿乙': ['我听到你这么说， 也很难过', '亿乙不应该让你这么难过的'],
    '亿甲难过亿乙': ['我听到你这么说， 也很难过',
                 '不应该让你这么难过的，你觉得你拥有什么，就会不难过?',
                 '你觉得事情变成什么样，你就不难过了?'],
    '亿甲就像亿乙': ['你觉得亿甲和亿乙有什么相似性？', '亿甲和亿乙真的有关系吗？', '怎么说？'],
    '亿甲和亿乙都?*z': ['你觉得?z有什么问题吗?', '?z会对你有什么影响呢?'],
    '亿甲和亿乙一样?*z': ['你觉得?z有什么问题吗?', '?z会对你有什么影响呢?'],
    '亿甲我是亿乙': ['真的吗？', '亿甲想告诉你，或许我早就知道你是亿乙', '你为什么现在才告诉我你是亿乙'],
    '亿甲我是亿乙吗': ['如果你是亿乙会怎么样呢？', '你觉得你是亿乙吗', '如果你是亿乙，那一位着什么?'],
    '亿甲你是亿乙吗':  ['你为什么会对我是不是亿乙感兴趣?', '那你希望我是亿乙吗', '你要是喜欢， 我就会是亿乙'],
    '亿甲你是亿乙' : ['为什么你觉得我是亿乙'],
    '亿甲因为亿乙' : ['亿乙是真正的原因吗？', '你觉得会有其他原因吗?'],
    '亿甲我不能亿乙': ['你或许现在就能亿乙', '如果你能亿乙,会怎样呢？'],
    '亿甲我觉得亿乙': ['你经常这样感觉吗？', '除了到这个，你还有什么其他的感觉吗？'],
    '亿甲我亿乙你?*z': ['其实很有可能我们互相亿乙'],
    '亿甲你为什么不亿乙': ['你自己为什么不亿乙', '你觉得我不会亿乙', '等我心情好了，我就亿乙'],
    '亿甲好的亿乙': ['好的', '你是一个很正能量的人'],
    '亿甲嗯嗯亿乙': ['好的', '你是一个很正能量的人'],
    '亿甲不嘛亿乙': ['为什么不？', '你有一点负能量', '你说 不，是想表达不想的意思吗？'],
    '亿甲不要亿乙': ['为什么不？', '你有一点负能量', '你说 不，是想表达不想的意思吗？'],
    '亿甲有些人亿乙': ['具体是哪些人呢?'],
    '亿甲有的人亿乙': ['具体是哪些人呢?'],
    '亿甲某些人亿乙': ['具体是哪些人呢?'],
    '亿甲每个人亿乙': ['我确定不是人人都是', '你能想到一点特殊情况吗？', '例如谁？', '你看到的其实只是一小部分人'],
    '亿甲所有人亿乙': ['我确定不是人人都是', '你能想到一点特殊情况吗？', '例如谁？', '你看到的其实只是一小部分人'],
    '亿甲总是亿乙': ['你能想到一些其他情况吗?', '例如什么时候?', '你具体是说哪一次？', '真的---总是吗？'],
    '亿甲一直亿乙': ['你能想到一些其他情况吗?', '例如什么时候?', '你具体是说哪一次？', '真的---总是吗？'],
    '亿甲或许亿乙': ['你看起来不太确定'],
    '亿甲可能亿乙': ['你看起来不太确定'],
    '亿甲他们是亿乙吗？': ['你觉得他们可能不是亿乙？'],
    '亿甲': ['很有趣', '请继续', '我不太确定我很理解你说的, 能稍微详细解释一下吗?']
}

In [539]:
import jieba

tpl_vars = ['亿甲','亿乙']

jieba.add_word('亿甲', freq=1, tag=None)
jieba.add_word('亿乙', freq=1, tag=None)

def is_chinese(ch):
    return '\u4e00' <= ch <= '\u9fa5'

# 定义多字匹配占位符格式 ?*+字
def is_pattern_segment(pattern):
    return pattern.startswith('亿') and all(is_chinese(a) for a in pattern[1:])

fail = [True, None]
# 语句列表和模式列表是否匹配，若是则返回匹配内容。
# 支持识别多个占位符的对应关系
# 支持占位符匹配多个单词
# pattern saying 都是 list
def pat_match_with_seg(pattern, saying):
    if not pattern or not saying: return []
    
    pat = pattern[0]
    
    # 占位符匹配单个单词
    if is_variable(pat):
        return [(pat, saying[0])] + pat_match_with_seg(pattern[1:], saying[1:])
    # 占位符匹配多个单词
    elif is_pattern_segment(pat):
        match, index = segment_match(pattern, saying)
        return [match] + pat_match_with_seg(pattern[1:], saying[index:])
    elif pat == saying[0]:
        return pat_match_with_seg(pattern[1:], saying[1:])
    else:
        return fail
# 语句列表和模式列表是否匹配，若是则返回匹配内容。
# 支持识别一个占位符的对应关系
# 支持占位符匹配多个单词
# 支持返回匹配的多个单词数目
def segment_match(pattern, saying):
    seg_pat, rest = pattern[0], pattern[1:]
    seg_pat = seg_pat.replace('?*', '?')

    if not rest: return (seg_pat, saying), len(saying)    
    
    for i, token in enumerate(saying):
#         if rest[0] == token and is_match(rest[1:], saying[(i + 1):]):
        if rest[0] == token:
            return (seg_pat, saying[:i]), i
    
    return (seg_pat, saying), len(saying)

def is_match(rest, saying):
    if not rest and not saying:
        return True
    if not all(is_chinese(a) for a in rest[0]):
        return True
    if rest[0] != saying[0]:
        return False
    return is_match(rest[1:], saying[1:])

def no_match(question_template, saying):
    for var in tpl_vars:
        question_template = question_template.replace(var,'|')
    tpl_list = question_template.split('|')
    for tpl_item in tpl_list:
        if tpl_item not in saying:
            return True
    return False
    
def get_response(saying, rules):
    # 找到提问对应问题模板
    # 提取问题的变量及值
    for question_template in rules.keys():
        if no_match(question_template, saying):
            continue
        say_list = [word for word in jieba.cut(saying)]
        tpl_list = [word for word in jieba.cut(question_template)]
        print('找到对应模板：')
        print("问题=",say_list)
        print("问题模板=",tpl_list)

        placeholders = pat_match_with_seg(tpl_list, say_list)
        print('提取对应变量值：', placeholders)
        if len(placeholders) != 0 and placeholders[0] != True: #!= fail

            found_template = question_template
            found_placeholders = placeholders
            found_answers = rules[question_template]
            print('问题模板',found_template)
            print('待替换内容',found_placeholders)
            print('回答模板',found_answers)
            # 随机选择回答
            choice_answer = found_answers[random.randint(0,len(found_answers)-1)]
        #     print(choice_answer)
            # 替换回答的变量值为问题中提取的变量值
            choice_list = [word for word in jieba.cut(choice_answer)]
            print(choice_list)
            final_answer = subsitite(choice_list, pat_to_dict(found_placeholders))
        #     print(final_answer)
            final_answer = ' '.join(final_answer)
            return final_answer
    return '不知道你在说啥？'


get_response('山水就像兄弟', rule_responses)

找到对应模板：
问题= ['山水', '就', '像', '兄弟']
问题模板= ['亿甲', '就', '像', '亿乙']
提取对应变量值： [('亿甲', ['山水']), ('亿乙', ['兄弟'])]
问题模板 亿甲就像亿乙
待替换内容 [('亿甲', ['山水']), ('亿乙', ['兄弟'])]
回答模板 ['你觉得亿甲和亿乙有什么相似性？', '亿甲和亿乙真的有关系吗？', '怎么说？']
['亿甲', '和', '亿乙', '真的', '有', '关系', '吗', '？']


'山水 和 兄弟 真的 有 关系 吗 ？'

### 问题3

多设计一些模式，让这个程序变得更好玩，多和大家交流，看看大家有什么好玩的模式

### 问题4

1. 这样的程序有什么优点？有什么缺点？你有什么可以改进的方法吗？ 
  - 优点：支持自定义问答模板
  - 缺点：相同意思的问题要定义多个问答对，不支持模糊匹配
  - 改进：计算问题/回答的相似度，提供最相似的几个问题/回答，让用户能自己选择

2. 什么是数据驱动？数据驱动在这个程序里如何体现？
数据驱动就是用一套通用代码支持不同的数据输入，得到对应输出。体现在支持问答模板。

3. 数据驱动与 AI 的关系是什么？ 
数据驱动的模型是AI的一种实现方式。